In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
FLIC_TOKEN = "flic_11d3da28e403d182c36a3530453e290add87d0b4a40ee50f17611f180d47956f"
HEADERS = {"Flic-Token": FLIC_TOKEN}

BASE_URL = "https://api.socialverseapp.com"

In [ ]:
# Get All Viewed Posts (METHOD: GET):

allViewedPosts=requests.get(f"{BASE_URL}/posts/view?resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")

# Get All Liked Posts (METHOD: GET):

allLikedPosts=requests.get("https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")

# Get All Inspired posts (METHOD: GET):

allInspiredPosts=requests.get("https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")

# Get All Rated posts (METHOD: GET):

allRatedPosts=requests.get("https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")

In [ ]:
# Get All Posts (Header required*) (METHOD: GET):

allPosts=requests.get("https://api.socialverseapp.com/posts/summary/get?page=1&page_size=1000",headers=HEADERS)

# Get All Users (Header required*) (METHOD: GET):

allUsers=requests.get("https://api.socialverseapp.com/users/get_all?page=1&page_size=1000",headers=HEADERS)

In [ ]:
def convertData(data):
  return pd.DataFrame(data.json())

In [ ]:
allViewedPostsdf=convertData(allViewedPosts)
allLikedPostsdf=convertData(allLikedPosts)
allInspiredPostsdf=convertData(allInspiredPosts)
allRatedPostsdf=convertData(allRatedPosts)
allPostsdf=convertData(allPosts)
allUsersdf=convertData(allUsers)

In [ ]:
def save_csv(data,name):
  data.to_csv(name, index=False)

In [ ]:
save_csv(allViewedPostsdf,"allViewedPostsdf.csv")
save_csv(allLikedPostsdf,"allLikedPostsdf.csv")
save_csv(allInspiredPostsdf,"allInspiredPostsdf.csv")
save_csv(allRatedPostsdf,"allRatedPostsdf.csv")
save_csv(allPostsdf,"allPostsdf.csv")
save_csv(allUsersdf,"allUsersdf.csv")

In [ ]:
# Expand the 'posts' dictionary into separate columns
viewed_posts_expanded = pd.json_normalize(allViewedPostsdf['posts'])
liked_posts_expanded = pd.json_normalize(allLikedPostsdf['posts'])
inspired_posts_expanded = pd.json_normalize(allInspiredPostsdf['posts'])
rated_posts_expanded = pd.json_normalize(allRatedPostsdf['posts'])
posts_expanded = pd.json_normalize(allPostsdf['posts'])
users_expanded = pd.json_normalize(allUsersdf['users'])

In [ ]:
viewed_posts_expanded.info()
liked_posts_expanded.info()
inspired_posts_expanded.info()
rated_posts_expanded.info()
posts_expanded.info()
users_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         30 non-null     int64 
 1   post_id    30 non-null     int64 
 2   user_id    30 non-null     int64 
 3   viewed_at  30 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   int64 
 1   post_id   1000 non-null   int64 
 2   user_id   1000 non-null   int64 
 3   liked_at  1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           390 non-null    int64 
 1 

In [ ]:
save_csv(viewed_posts_expanded,"viewed_posts_expanded.csv")
save_csv(liked_posts_expanded,"liked_posts_expanded.csv")
save_csv(inspired_posts_expanded,"inspired_posts_expanded.csv")
save_csv(rated_posts_expanded,"rated_posts_expanded.csv")
save_csv(posts_expanded,"posts_expanded.csv")
save_csv(users_expanded,"users_expanded.csv")

In [ ]:
import pandas as pd

# Load the expanded CSVs
users = pd.read_csv("users_expanded.csv")
posts = pd.read_csv("posts_expanded.csv")
viewed = pd.read_csv("viewed_posts_expanded.csv")
liked = pd.read_csv("liked_posts_expanded.csv")
inspired = pd.read_csv("inspired_posts_expanded.csv")
rated = pd.read_csv("rated_posts_expanded.csv")

# Merge interactions with posts
viewed_merged = viewed.merge(posts, left_on='post_id', right_on='id', suffixes=('_viewed', '_post'))
liked_merged = liked.merge(posts, left_on='post_id', right_on='id', suffixes=('_liked', '_post'))
inspired_merged = inspired.merge(posts, left_on='post_id', right_on='id', suffixes=('_inspired', '_post'))
rated_merged = rated.merge(posts, left_on='post_id', right_on='id', suffixes=('_rated', '_post'))

# Merge interactions with users
viewed_merged = viewed_merged.merge(users, left_on='user_id', right_on='id', suffixes=('', '_user'))
liked_merged = liked_merged.merge(users, left_on='user_id', right_on='id', suffixes=('', '_user'))
inspired_merged = inspired_merged.merge(users, left_on='user_id', right_on='id', suffixes=('', '_user'))
rated_merged = rated_merged.merge(users, left_on='user_id', right_on='id', suffixes=('', '_user'))

# Save the merged datasets
viewed_merged.to_csv("merged_viewed.csv", index=False)
liked_merged.to_csv("merged_liked.csv", index=False)
inspired_merged.to_csv("merged_inspired.csv", index=False)
rated_merged.to_csv("merged_rated.csv", index=False)


<ipython-input-18-5d6cf31a79aa>:5: DtypeWarning: Columns (14,15,16,29,39,40,41,42,49,52,61,63,67,70,71,72,81,117,118,119,121,133,134,138,150,151,157,158,159,160,161,162,163,164,167,171,172,176,177,178,182,183,184,192,194,196,201,206,208,210,211,212,213,215,216,223,224,225,226,229,235,236,237,238,240,243,245,247,248,251,252,253,254,257,258,259,262,263,268,271,272,274,275,276,277,278,279,280,281,282,283,284,286,290,291,295,296,297,298,301,304,305,307,309,310,311,318,319,325,327,328,329,330,331,332,333,334,335,338,341,344,345,346,347,348,349,350,354,355,356,357,358,359,360,361,362,364,365,370,371,380,381,382,383,384,387,388,391,392,393,395,397,398,399,400,401,402,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,461,466,469,470,472,473,474,476,479,480,481,482,485,486,489,490,493,494,495,496,497,498,499,500,506,507,508,509,510

In [ ]:
liked_merged.columns

Index(['id_liked', 'post_id', 'user_id', 'liked_at', 'id_post', 'topic',
       'slug', 'title', 'identifier', 'comment_count',
       ...
       'referral_count', 'total_inspired_user_count', 'daily_login_streak',
       'post_count', 'following_count', 'follower_count', 'is_verified',
       'is_online', 'latitude', 'longitude'],
      dtype='object', length=1741)

In [ ]:
# Load merged liked dataset
liked_merged = pd.read_csv("merged_liked.csv")

# Define important columns
important_columns = [
    'post_id', 'user_id', 'title', 'comment_count',
    'view_count', 'exit_count', 'rating_count',
    'average_rating', 'post_count',
    'follower_count', 'following_count'
]

# Filter dataset
liked_filtered = liked_merged[important_columns]

# Save cleaned dataset
liked_filtered.to_csv("liked_filtered.csv", index=False)


<ipython-input-20-6ac72a37d5e9>:2: DtypeWarning: Columns (52,53,64,65,67,85,87,106,122,123,124,125,127,130,134,138,142,143,144,151,154,158,162,163,164,165,166,167,168,171,175,176,179,180,181,182,186,187,188,191,192,201,204,205,207,210,212,213,214,215,216,217,219,220,223,224,225,226,227,228,229,232,233,235,237,239,240,245,248,255,256,257,258,268,269,270,271,279,296,302,303,306,307,310,313,314,315,316,320,324,325,326,327,330,336,341,347,357,370,371,372,373,374,375,377,378,379,380,381,382,383,384,385,386,387,388,390,391,396,399,402,406,442,464,466,468,471,472,474,482,485,487,491,492,496,506,507,510,513,515,520,528,529,535,536,537,544,546,559,560,561,562,563,564,565,566,567,568,569,570,572,574,575,576,577,578,579,580,581,582,583,584,586,587,588,589,590,591,592,594,596,597,602,603,604,605,606,607,608,610,611,612,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,636,637,639,640,641,642,643,644,647,648,649,650,651,652,653,654,655,656,657,661,670,671,673,674,675,682,7

In [ ]:
liked_filtered['liked'] = 1  # All are positive interactions


<ipython-input-21-1296ccab9821>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liked_filtered['liked'] = 1  # All are positive interactions


In [ ]:
liked_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   post_id          1000 non-null   int64 
 1   user_id          1000 non-null   int64 
 2   title            677 non-null    object
 3   comment_count    1000 non-null   int64 
 4   view_count       1000 non-null   int64 
 5   exit_count       1000 non-null   int64 
 6   rating_count     1000 non-null   int64 
 7   average_rating   1000 non-null   int64 
 8   post_count       1000 non-null   int64 
 9   follower_count   1000 non-null   int64 
 10  following_count  1000 non-null   int64 
 11  liked            1000 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 93.9+ KB


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols_to_scale = ['comment_count', 'view_count', 'exit_count',
                 'rating_count', 'average_rating',
                 'post_count', 'follower_count', 'following_count']

liked_filtered[cols_to_scale] = scaler.fit_transform(liked_filtered[cols_to_scale])


<ipython-input-23-2795ce3e5633>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liked_filtered[cols_to_scale] = scaler.fit_transform(liked_filtered[cols_to_scale])


In [ ]:
liked_filtered.to_csv("liked_filtered2.csv", index=False)

In [ ]:
liked_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   post_id          1000 non-null   int64  
 1   user_id          1000 non-null   int64  
 2   title            677 non-null    object 
 3   comment_count    1000 non-null   float64
 4   view_count       1000 non-null   float64
 5   exit_count       1000 non-null   float64
 6   rating_count     1000 non-null   float64
 7   average_rating   1000 non-null   float64
 8   post_count       1000 non-null   float64
 9   follower_count   1000 non-null   float64
 10  following_count  1000 non-null   float64
 11  liked            1000 non-null   int64  
dtypes: float64(8), int64(3), object(1)
memory usage: 93.9+ KB


In [ ]:
liked_filtered.dropna(inplace=True)

<ipython-input-26-c00343806316>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liked_filtered.dropna(inplace=True)


In [ ]:
# Add 'liked' column if not already present
liked_filtered['liked'] = 1

# Randomly select 250 indices to flip
np.random.seed(42)  # For reproducibility
negative_indices = np.random.choice(liked_filtered.index, size=250, replace=False)

# Flip those 'liked' values to 0
liked_filtered.loc[negative_indices, 'liked'] = 0

# Save the modified dataset
liked_filtered.to_csv("balanced_liked_dataset.csv", index=False)


<ipython-input-27-747e9d3954f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liked_filtered['liked'] = 1


In [ ]:
liked_filtered=pd.read_csv("/content/liked_filtered_2.csv")
# Add 'liked' column if not already present
liked_filtered['liked'] = 1

# Randomly select 250 indices to flip
np.random.seed(42)  # For reproducibility
negative_indices = np.random.choice(liked_filtered.index, size=250, replace=False)

# Flip those 'liked' values to 0
liked_filtered.loc[negative_indices, 'liked'] = 0

# Save the modified dataset
liked_filtered.to_csv("balanced_liked_dataset2.csv", index=False)
